In [1]:
# Imports
import time
import cv2
import numpy as np
from PIL import Image

In [2]:
def chromakey_manual(background: np.array, foreground: np.array, key_color: tuple, threshold: int = 50):
    """
    Manual implementation of chromakey.
        :param background: Background image as numpy array.
        :param foreground: Foreground image as numpy array.
        :param key_color: The RGB color to replace (tuple).
        :param threshold: Color similarity threshold.
        :return: Composited image.
    """
    if foreground.shape[:2] != background.shape[:2]:
        raise ValueError("The foreground and background must have the same dimensions.")
    
    result = np.copy(background)
    rows, cols, _ = foreground.shape

    for y in range(rows):
        for x in range(cols):
            fg_pixel = foreground[y, x]
            distance = np.linalg.norm(fg_pixel - key_color)
            if distance > threshold:
                result[y, x] = fg_pixel

    return result

In [3]:
def chromakey_opencv(background: np.ndarray, foreground: np.ndarray, key_color: tuple, threshold: int = 50):
    """
    Chromakey implementation using OpenCV.
        :param background: Background image as numpy array.
        :param foreground: Foreground image as numpy array.
        :param key_color: The RGB color to replace (tuple).
        :param threshold: Color similarity threshold.
        :return: Composited image.
    """
    if foreground.shape[:2] != background.shape[:2]:
        raise ValueError("The foreground and background must have the same dimensions.")
    
    lower_bound = np.array([channel - threshold for channel in key_color])
    upper_bound = np.array([channel + threshold for channel in key_color])

    mask = cv2.inRange(foreground, lower_bound, upper_bound)
    mask_inv = cv2.bitwise_not(mask)

    fg_masked = cv2.bitwise_and(foreground, foreground, mask=mask_inv)
    bg_masked = cv2.bitwise_and(background, background, mask=mask)

    result = cv2.add(bg_masked, fg_masked)
    return result

In [4]:
foreground_path = 'images/foreground.jpg'
background_path = 'images/background.jpg'

foreground = np.array(Image.open(foreground_path).convert("RGB"))
background = np.array(Image.open(background_path).convert("RGB"))
key_color = (255, 255, 255)  # White
threshold = 50

start_time = time.time()
result_manual = chromakey_manual(background, foreground, key_color, threshold)
manual_time = time.time() - start_time

start_time = time.time()
result_cv2 = chromakey_opencv(background, foreground, key_color, threshold)
cv2_time = time.time() - start_time

manual_image = Image.fromarray(result_manual)
cv2_image =  Image.fromarray(result_cv2)

manual_image.save("results/manual.png")
cv2_image.save("results/openCV.png")

print(f'Manual time: {manual_time:.6f} seconds\nOpenCV time: {cv2_time:.6f} seconds')

Manual time: 0.621435 seconds
OpenCV time: 0.002212 seconds
